<a href="https://colab.research.google.com/github/ReemFarah/ReemFarah.github.io/blob/main/CC8_Big_Data_Extracting_a_story_from_millions_of_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

In [ ]:
prices_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/redacted_prices_df.csv')
items_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/redacted_items_df.csv')

In [ ]:
print(prices_df.head())
print(prices_df.info())
print(items_df.head())
print(items_df.info())

         date  price      unit_price  loyalty_price  original_price  store_id  \
0  2023-10-06  12.95  0.16 per 100ml            NaN             NaN         5   
1  2023-10-06   9.00      9 per 75cl            NaN             NaN         5   
2  2023-10-06   4.00   1.29 per 100g            NaN             NaN         5   
3  2023-10-06   2.50   1.67 per 100g           2.00             NaN         5   
4  2023-10-06   1.50   1.36 per 100g           1.35             NaN         5   

   product_id  
0    209870.0  
1    265800.0  
2    181052.0  
3    122275.0  
4    164794.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484040 entries, 0 to 5484039
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   price           float64
 2   unit_price      object 
 3   loyalty_price   float64
 4   original_price  float64
 5   store_id        int64  
 6   product_id      float64
dtypes: float64(4), int64(1), object(2)
memo

In [ ]:
# Merge datasets on product_id (or equivalent matching column)
merged_data = pd.merge(prices_df, items_df, on='product_id', how='inner')

# Preview the merged data
print(merged_data.head())

         date  price      unit_price  loyalty_price  original_price  \
0  2023-10-06  12.95  0.16 per 100ml            NaN             NaN   
1  2023-10-06  12.95  0.16 per 100ml            NaN             NaN   
2  2023-10-06   9.00      9 per 75cl            NaN             NaN   
3  2023-10-06   4.00   1.29 per 100g            NaN             NaN   
4  2023-10-06   4.00   1.29 per 100g            NaN             NaN   

   store_id_x  product_id  store_id_y    cpi_id  \
0           5    209870.0           5  212023.0   
1           5    209870.0           5  212025.0   
2           5    265800.0           5  310426.0   
3           5    181052.0           5  212228.0   
4           5    181052.0           5  212218.0   

                         cpi_name  
0   cola/fizzy drink 330ml pk 4-8  
1  cola drink, reg,bottle,1.25-2l  
2  sparkling wine 75cl min 11%abv  
3         malted chocolate sweets  
4   carton/box of chocs 150-400gm  


In [ ]:
# Filter for a specific category (e.g., products containing "cola drink")
specific_category = merged_data[merged_data['cpi_name'].str.contains("cola drink", case=False, na=False)]

# Group data by date and calculate the average price for the category
category_price_trends = (
    specific_category.groupby('date')['price']
    .mean()
    .reset_index()
)

# Sort by date to ensure proper time series plotting
category_price_trends = category_price_trends.sort_values(by='date')

In [ ]:
print(merged_data.columns)

Index(['date', 'price', 'unit_price', 'loyalty_price', 'original_price',
       'store_id_x', 'product_id', 'store_id_y', 'cpi_id', 'cpi_name'],
      dtype='object')


In [ ]:
# Group data by store and calculate average prices
store_prices = (
    merged_data.groupby('store_id_y')['price']
    .mean()
    .reset_index()
)

# Sort by price for better visualization
store_prices = store_prices.sort_values(by='price', ascending=False)

# Display the resulting DataFrame to verify the output
print(store_prices)

   store_id_y     price
1           2  7.716530
2           3  5.225812
5           6  5.032094
4           5  4.833674
0           1  4.357184
6           7  3.687768
3           4  2.280244


In [ ]:
import altair as alt
import pandas as pd

# Assuming `category_price_trends` is a Pandas DataFrame
# Replace `category_price_trends` with your actual DataFrame variable
# Example DataFrame structure:
# category_price_trends = pd.DataFrame({
#     'date': ['2023-01-01', '2023-01-02', '2023-01-03'],
#     'price': [1.5, 1.6, 1.4]
# })

# Convert date column to datetime if it's not already
category_price_trends['date'] = pd.to_datetime(category_price_trends['date'])

# Create the Altair chart
chart = alt.Chart(category_price_trends).mark_line(color='dodgerblue', strokeWidth=2).encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('price:Q', title='Average Price (£)'),
    tooltip=['date:T', 'price:Q']
).properties(
    title='Average Price Trends Over Time for Cola Drinks',
    width=600,
    height=400
)

# Display the chart
chart.show()


alt.Chart(...)

In [ ]:
import altair as alt
import pandas as pd

# Assuming `category_price_trends` is a Pandas DataFrame
# Convert date column to datetime if it's not already
category_price_trends['date'] = pd.to_datetime(category_price_trends['date'])

# Create the Altair chart
chart = alt.Chart(category_price_trends).mark_line(color='dodgerblue', strokeWidth=2).encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('price:Q', title='Average Price (£)', scale=alt.Scale(domain=[3, 6])),  # Adjust y-axis scale
    tooltip=['date:T', 'price:Q'],
    color=alt.value('dodgerblue')
).properties(
    title='Average Price Trends Over Time for Cola Drinks',
    width=800,
    height=500
).interactive()

# Add a built-in legend to indicate "Cola Drinks"
chart = chart.encode(
    color=alt.value('dodgerblue')  # Keeps the color fixed
).properties(
    title={
        "text": ["Average Price Trends Over Time for Cola Drinks"],
        "subtitle": ["Category: Cola Drinks"]
    }
)

# Display the final chart
chart.show()


alt.Chart(...)

This chart shows the average price trends for a specific product category (e.g., 'Dairy') over time. Aggregating daily prices highlights temporal price variations and seasonal effects, offering insights into market behaviour and price fluctuations.

In [ ]:
import altair as alt
import pandas as pd

# Assuming `store_prices` is a Pandas DataFrame
# Example DataFrame structure:
# store_prices = pd.DataFrame({
#     'store_id_y': ['Store 1', 'Store 2', 'Store 3'],
#     'price': [3.5, 4.0, 3.8]
# })

# Create the Altair bar chart
chart = alt.Chart(store_prices).mark_bar(color='dodgerblue').encode(
    x=alt.X('store_id_y:N', title='Store ID', sort=None),  # Nominal data for x-axis
    y=alt.Y('price:Q', title='Average Price (£)'),        # Quantitative data for y-axis
    tooltip=['store_id_y', 'price']                        # Add tooltips for interactivity
).properties(
    title='Average Prices by Store',
    width=800,                                            # Set the width of the chart
    height=500                                            # Set the height of the chart
).configure_axis(
    grid=False                                             # Disable x-axis grid
).configure_view(
    strokeWidth=0                                          # Remove the border around the chart
)

# Display the chart
chart.show()


alt.Chart(...)

In [ ]:
import altair as alt

# Assuming `store_prices` is your DataFrame in Colab and contains accurate data

# Create the Altair bar chart with explicit formatting
chart = alt.Chart(store_prices).mark_bar(color='dodgerblue').encode(
    x=alt.X('store_id_y:N', title='Store ID', sort=None),  # Nominal data for Store IDs
    y=alt.Y('price:Q', title='Average Price (£)', axis=alt.Axis(format=".1f")),  # Format y-axis to 1 decimal place
    tooltip=[
        alt.Tooltip('store_id_y:N', title='Store ID'),
        alt.Tooltip('price:Q', title='Average Price (£)', format=".1f")  # Format tooltip to 1 decimal place
    ]
).properties(
    title='Average Prices by Store',
    width=800,                                            # Set the width of the chart
    height=500                                            # Set the height of the chart
).configure_axis(
    grid=True,                                            # Enable grid for y-axis
    labelFontSize=12,                                     # Adjust font size for readability
    titleFontSize=14
).configure_view(
    strokeWidth=0                                         # Remove the border around the chart
)

# Display the chart
chart.show()


alt.Chart(...)

This bar chart compares the average prices of products across different stores. It emphasises price disparities among retailers, aiding in identifying stores with consistently higher or lower pricing strategies.